In [15]:
URL = "http://www.jhowell.net/cf/scores/Sked2017.htm"

In [16]:
import urllib
import string
import re
import operator
from bs4 import BeautifulSoup

In [25]:
with urllib.request.urlopen(URL) as response:
   response = response.read()

In [28]:
#Page is cached. Uncomment to reload
page = BeautifulSoup(response, 'html.parser')

In [31]:
page

<html><head><title>2017 Division IA Schedules and Results</title></head>
<body><p><a href="http://www.jhowell.net/cf/scores/ScoresIndex.htm">Return to the all-time scores index</a></p>
<p>Listed below are the Division IA schedules and results for 2017.
<br/>This file was last updated on Sunday, October 22, 2017.
<br/>Conference affiliation is in parenthesis. Conference games are prefixed with an asterisk.</p>
 
<table border="1">
<tr><td bgcolor="#COCOCO" colspan="9"><p align="center">Air Force (MWC)</p></td></tr>
<tr><td align="right">9/2</td><td>Sat</td><td>vs.</td><td>Virginia Military Institute</td><td>W</td><td align="right">62</td><td align="right">0</td></tr>
<tr><td align="right">9/16</td><td>Sat</td><td>@</td><td>Michigan</td><td>L</td><td align="right">13</td><td align="right">29</td></tr>
<tr><td align="right">9/23</td><td>Sat</td><td>vs.</td><td>*San Diego State</td><td>L</td><td align="right">24</td><td align="right">28</td></tr>
<tr><td align="right">9/30</td><td>Sat</td>

Functions to Extract Games

In [32]:
def get_games(team):
    return team.find_all('tr')[1:]

In [33]:
def get_team_1(team):
    try:
        return team.find_all('p')[0].text
    except:
        return None

In [34]:
def get_score_1(game):
    try:
        score = int(game.find_all('td')[5].text)
        return score
    except:
        return None

In [35]:
def get_score_2(game):
    try:
        score = int(game.find_all('td')[6].text)
        return score
    except:
        return None

In [74]:
def get_opponent(game):
    try:
        opp = game.find_all('td')[3].text
        opp = opp.replace('*', '')
        return opp
    except:
        return None

In [75]:
def del_conf(team_name):
    my_re = re.compile(r"( \(\w\w+ \w+\)$)|( \(\w+\)$)")
    try:
        return re.sub(my_re, '', team_name)
    except:
        return team_name

In [76]:
def get_game_details(game, team1):
    if get_score_1(game):
        return (del_conf(team1), del_conf(get_opponent(game)), get_score_1(game), get_score_2(game))
    else:
        return

In [77]:
def parse_games(team):
    team1 = get_team_1(team)
    games = [get_game_details(game, team1) for game in get_games(team)[:-1] \
             if get_game_details(game, team1)]
    return games

In [78]:
teams = page.find_all('table')

In [79]:
team1 = get_team_1(teams[0])

In [80]:
#games = [get_game_details(game, team1) for game in get_games(team)[:-1] if get_game_details(game, team1)]

In [81]:
my_game = get_games(team)[0]

In [82]:
my_game.find_all('td')[3].text

'Iowa'

In [83]:
get_opponent(my_game)

'Iowa'

In [84]:


teams.findall('')

AttributeError: ResultSet object has no attribute 'findall'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

#### Now, get all the games

In [85]:
all_games = []
for team in teams:
    all_games.extend(parse_games(team))

In [86]:
len(all_games)

920

In [87]:
all_games

[('Air Force', 'Virginia Military Institute', 62, 0),
 ('Air Force', 'Michigan', 13, 29),
 ('Air Force', 'San Diego State', 24, 28),
 ('Air Force', 'New Mexico', 38, 56),
 ('Air Force', 'Navy', 45, 48),
 ('Air Force', 'Nevada-Las Vegas', 34, 30),
 ('Air Force', 'Nevada', 45, 42),
 ('Akron', 'Arkansas-Pine Bluff', 52, 3),
 ('Akron', 'Iowa State', 14, 41),
 ('Akron', 'Troy', 17, 22),
 ('Akron', 'Bowling Green State', 34, 23),
 ('Akron', 'Ball State', 31, 3),
 ('Akron', 'Western Michigan', 14, 13),
 ('Akron', 'Toledo', 21, 48),
 ('Alabama', 'Florida State', 24, 7),
 ('Alabama', 'Fresno State', 41, 10),
 ('Alabama', 'Colorado State', 41, 23),
 ('Alabama', 'Vanderbilt', 59, 0),
 ('Alabama', 'Mississippi', 66, 3),
 ('Alabama', 'Texas A&M;', 27, 19),
 ('Alabama', 'Arkansas', 41, 9),
 ('Alabama', 'Tennessee', 45, 7),
 ('Alabama-Birmingham', 'Alabama A&M;', 38, 7),
 ('Alabama-Birmingham', 'Ball State', 31, 51),
 ('Alabama-Birmingham', 'Coastal Carolina', 30, 23),
 ('Alabama-Birmingham', 'North 

In [88]:
example

'*New Mexico'

In [89]:
example = '*New Mexico'
example2 = 'New Mexico'

In [90]:
example2.replace('*', '')

'New Mexico'

There are approximately twice as many games as there should be because the teams appear in both orders

In [91]:
def alpha(game):
    team1, team2, score1, score2 = game
    if team1 < team2:
        return game
    else:
        return (team2, team1, score2, score1)

In [92]:
all_games_alpha = [alpha(game) for game in all_games]
all_games_alpha = list(set(all_games_alpha))

In [93]:
len(all_games_alpha)

525

It works!

### Random Walk Model

In [94]:
import networkx as nx

First, prepare the edges.

In [95]:
def game_to_edge_ratio(game):
    team1, team2, score1, score2 = game
    if score1 <= score2:
        return (team1, team2, np.sqrt(float(score2-score1)/score2))
    else:
        return (team2, team1, np.sqrt(float(score1-score2)/score1))

In [96]:
def game_to_edge(game):
    team1, team2, score1, score2 = game
    if score1 <= score2:
        return (team1, team2, float(score2-score1))
    else:
        return (team2, team1, float(score1-score2))

In [97]:
edge_list = [game_to_edge(game) for game in all_games_alpha]

Sanity Check: Did I get all the games with no repeats?

In [99]:
print(len([edge for edge in edge_list if edge[0]=='Michigan' or edge[1]=='Michigan']))
[edge for edge in edge_list if edge[0]=='Michigan' or edge[1]=='Michigan']

7


[('Cincinnati', 'Michigan', 22.0),
 ('Florida', 'Michigan', 16.0),
 ('Air Force', 'Michigan', 16.0),
 ('Michigan', 'Michigan State', 4.0),
 ('Indiana', 'Michigan', 7.0),
 ('Purdue', 'Michigan', 18.0),
 ('Michigan', 'Penn State', 29.0)]

In [100]:
G = nx.DiGraph()

In [101]:
G.add_weighted_edges_from(edge_list)

In [102]:
ranked = nx.pagerank_numpy(G, alpha=.65)

I'll need the teams alphabetically sorted later...

In [103]:
page_rank_sorted = sorted(ranked.items(), key=operator.itemgetter(0))

But let's look at what the teams look like sorted by the random walk ranking.

In [104]:
sorted(ranked.items(), key=operator.itemgetter(1))[:-20:-1]

[('Georgia', 0.027556343871767433),
 ('Notre Dame', 0.02243333768137143),
 ('Clemson', 0.017777368984828377),
 ('Alabama', 0.01663683305561863),
 ('Mississippi State', 0.01598109763849987),
 ('Iowa State', 0.015928893782690624),
 ('Syracuse', 0.015827012920808167),
 ('Texas Christian', 0.0157777689982492),
 ('Oklahoma', 0.014337028804769012),
 ('Arizona State', 0.013354335517201641),
 ('Penn State', 0.012652002208014074),
 ('Central Florida', 0.012547097675031605),
 ('Auburn', 0.011972696838047834),
 ('Southern California', 0.011960887365065584),
 ('Washington', 0.011781285359938058),
 ('Ohio State', 0.01145701062589429),
 ('Texas', 0.010972944784021657),
 ('North Carolina State', 0.010670798095622675),
 ('South Carolina', 0.010295092293857461)]

### Regression Model

In [105]:
import numpy as np

In [106]:
all_teams = set()
for team1, team2, _, _ in all_games_alpha:
    all_teams.add(team1)
    all_teams.add(team2)
all_teams = list(all_teams)

I want to make a matrix with a row for each game. The teams will be the columns. I'll assign each team a number that will correspond to the column index.

In [107]:
all_teams_tuples = zip(all_teams, range(len(all_teams)))
team_dict = dict(all_teams_tuples)

In [108]:
my_matrix = np.empty((0,len(all_teams)), int)
my_vector = np.empty((0, 1))

In [109]:
for team1, team2, score1, score2 in all_games_alpha:
    row = np.zeros(len(all_teams))
    row[team_dict[team1]]=1
    row[team_dict[team2]]=-1
    my_matrix = np.append(my_matrix, [row], axis=0)
    my_vector = np.append(my_vector, [[score1 - score2]], axis=0)

#adding the Massey Fudge Factor
my_matrix = np.append(my_matrix, [np.ones(len(all_teams))], axis=0)
my_vector = np.append(my_vector, [[0]], axis=0)

Sanity check: Do my matrix and vector have compatible dimensions?

In [111]:
print(my_matrix.shape)
print(my_vector.shape)

(526, 210)
(526, 1)


Least Squares -- my favorite thing

In [112]:
soln = np.linalg.lstsq(my_matrix, my_vector)

In [113]:
ranks = list(np.ndarray.flatten(soln[0]))

In [114]:
ranked = zip(all_teams, ranks)

In [115]:
sorted_x = sorted(ranked, key=operator.itemgetter(1))

Let's see the results

In [116]:
sorted_x[:-20:-1]

[('Alabama', 45.928740962811588),
 ('Ohio State', 44.917722389827318),
 ('Penn State', 44.416209454554206),
 ('Notre Dame', 43.297013876086375),
 ('Clemson', 40.680722081124543),
 ('Georgia', 40.071158174643969),
 ('Central Florida', 36.452915824755998),
 ('Wisconsin', 36.181800714614582),
 ('Oklahoma State', 35.887366427495792),
 ('Virginia Tech', 35.842868735008359),
 ('Texas Christian', 35.780378630715468),
 ('Auburn', 35.215588406437185),
 ('Oklahoma', 33.262606032530364),
 ('Washington', 32.879365223288588),
 ('Western Illinois', 30.792069743221884),
 ('Iowa State', 30.050840263547421),
 ('Stanford', 28.265258991215859),
 ('Mississippi State', 27.771213016342514),
 ('Texas', 26.482788051338609)]

Portland State! Are you kidding me? I'll use regularization to correct this apparent over-fitting.

### Regularized Regression

In [117]:
from sklearn import linear_model, grid_search, preprocessing

/Users/danielfile/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/danielfile/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [118]:
lm = linear_model.Ridge()
parameters = {'alpha':[0.01, 0.1, 1, 10, 100]}


In [119]:
clf = grid_search.GridSearchCV(lm, parameters)

In [120]:
clf.fit(my_matrix, my_vector)

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [121]:
ridge = clf.best_estimator_

In [122]:
ridge.fit(my_matrix, my_vector)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

I have my model, and it's been fited. Let's take a look.

In [123]:
linear_rank = zip(all_teams, list(preprocessing.scale(ridge.coef_[0])))

In [124]:
sorted(linear_rank, key=operator.itemgetter(1))[:-20:-1]

[('Alabama', 2.2425768107302062),
 ('Ohio State', 2.1298733634801987),
 ('Penn State', 2.0769853711764101),
 ('Notre Dame', 2.0116652569112725),
 ('Clemson', 1.924603532851985),
 ('Georgia', 1.9122442408077609),
 ('Central Florida', 1.7758937039730498),
 ('Oklahoma State', 1.7052534881402999),
 ('Auburn', 1.6901581816996316),
 ('Texas Christian', 1.6765027236088641),
 ('Wisconsin', 1.6619679564279919),
 ('Virginia Tech', 1.6317688405544515),
 ('Oklahoma', 1.5469178137071016),
 ('Washington', 1.5417165187443449),
 ('Iowa State', 1.4138226793246269),
 ('Western Illinois', 1.353158923704751),
 ('Stanford', 1.2939463844307979),
 ('Mississippi State', 1.2792096689375778),
 ('Miami (Florida)', 1.2207888518924985)]

It knocked down Portland State a little bit, but not much...

### Ensemble

I need to put the two models on the same scale.

In [125]:
linear_rank_sorted = sorted(linear_rank, key=operator.itemgetter(0))

In [126]:
page_ranks = list(preprocessing.scale(zip(*page_rank_sorted)[1]))

TypeError: 'zip' object is not subscriptable

In [127]:
final = zip(linear_rank_sorted, page_ranks)

NameError: name 'page_ranks' is not defined

In [ ]:
final = [(team, first + second) for (team, first), second in final]

In [ ]:
final = sorted(final, key=operator.itemgetter(1))

In [ ]:
final[:-50:-1]